In [ ]:
import csv 
import glob
import os

import astropy.units as u
from astropy.table import Table, Column
from astropy.io import ascii
from astropy.io.ascii import Csv
from astropy.io.votable.tree import Param,Info
from astropy.io.votable import from_table, writeto
from astropy.io import fits, votable
from astropy.wcs import WCS
from astropy.table import QTable
import matplotlib.pyplot as plt
import numpy as np
import numpy.core.records as rec

from spectral_cube import SpectralCube
import radio_beam
import aplpy
import astropy.units as u


from RadioAbsTools import cube_tools, spectrum_tools



# Extract many absorpton spectra from ASKAP SMC Data

This notebook produces spectra for each cutout subcube proved for SB 8906.

In [ ]:
def output_spectra(velocity, opacity, flux, filename, 
                   sigma_tau):
    """
    Write the spectrum (velocity, flux and opacity) to a votable format file.

    :param spectrum: The spectrum to be output.
    :param opacity:  The opacity to be output.
    :param filename:  The filename to be created
    :param longitude: The galactic longitude of the target object
    :param latitude: The galactic latitude of the target object
    """
    table = Table(meta={'name': filename, 'id': 'opacity'})
    table.add_column(Column(name='velocity', data=velocity, unit='m/s'))
    table.add_column(Column(name='opacity', data=opacity))
    table.add_column(Column(name='flux', data=spectrum.flux, unit='Jy', description='Flux per beam'))
    table.add_column(Column(name='sigma_opacity', data=sigma_tau, description='Noise in the absorption profile'))

    votable = from_table(table)
    writeto(votable, filename)



In [ ]:
def read_targets():
    ids=[]
    comp_names=[]
    ras=[]
    decs=[]
    beams=[]
    i=1
    with open('targets.csv', 'r') as csvfile:
        tgt_reader = csv.reader(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in tgt_reader:
            if (tgt_reader.line_num == 1):
                # skip header
                continue

            #print (row)
            ids.append(i)
            comp_names.append(row[1])
            ras.append(float(row[2]))
            decs.append(float(row[3]))
            beams.append(row[4:])

            i+=1

            
    table = Table()
    table.add_column(Column(name='id', data=ids))
    table.add_column(Column(name='comp_name', data=comp_names))
    table.add_column(Column(name='ra', data=ras))
    table.add_column(Column(name='dec', data=decs))
    table.add_column(Column(name='beams', data=beams))

    return table



In [ ]:
def plot_mom0(fname, comp_name, out_folder, src_ra, src_dec, src_maj, src_min, src_pa):
    cube = SpectralCube.read(fname)
    cube.beam_threshold = 0.1
    m0 = cube.moment0().to(u.Jy*u.km/(u.beam*u.s))
    m0.write('moment-0.fits', overwrite=True)

    fig = aplpy.FITSFigure('moment-0.fits')
    fig.show_grayscale()#pmax=100)
    fig.add_colorbar()
    fig.colorbar.set_axis_label_text('Brightness (Jy km / s / beam)')
    fig.add_beam()
    fig.set_theme('publication')

    # Plot ellipse for each source
    a_deg = src_maj*u.arcsec.to(u.deg)
    b_deg = src_min*u.arcsec.to(u.deg)
    fig.show_ellipses(src_ra, src_dec, a_deg, b_deg, angle=src_pa, edgecolor='red') #, 

    figname = '{}/{}_mom0.'.format(out_folder, comp_name)
    fig.savefig(figname+'.pdf')
    fig.savefig(figname+'.png')


### Identify the sources that we have cutouts for

In [ ]:
folder = 'sb8906/cutouts/'
file_list = glob.glob(folder + 'J*_sl.fits')
file_list.sort()
len(file_list)

In [ ]:
max_spectra=1000
#targets = ascii.read('targets.csv', format='csv', quotechar='"')
targets = read_targets()
targets

In [ ]:
# Read and filter catalogue
src_votable = votable.parse('AS037_Continuum_Component_Catalogue_8906_100.votable', pedantic=False)
selavy_table = src_votable.get_first_table().to_table()
selavy_table

### For each cutout:
1. Get source params from Selavy catalogue
2. Produce moment map
3. Produce plot of moment map with source highlighted
4. Extract spectrum for source
5. Classify spectrum
6. Save to votable file

In [ ]:
def get_source(file_list, target):
    
    fname = '{}{}_sl.fits'.format(folder, tgt['comp_name'])
    if not fname in file_list:
        return None
    
    comp_cat_row= selavy_table[selavy_table['component_name']==target['comp_name']][0]
    src = {'ra':comp_cat_row['ra_deg_cont'], 'dec':comp_cat_row['dec_deg_cont'], 
           'a':comp_cat_row['maj_axis'], 'b':comp_cat_row['min_axis'], 'pa': comp_cat_row['pos_ang'],
          'comp_name': target['comp_name'], 'fname': fname, 'flux_peak': comp_cat_row['flux_peak']}
    return src
    

In [ ]:
def save_spectrum(velocity, opacity, flux, filename, 
                   sigma_tau):
    """
    Write the spectrum (velocity, flux and opacity) to a votable format file.

    :param spectrum: The spectrum to be output.
    :param opacity:  The opacity to be output.
    :param filename:  The filename to be created
    :param longitude: The galactic longitude of the target object
    :param latitude: The galactic latitude of the target object
    """
    table = Table(meta={'name': filename, 'id': 'opacity'})
    table.add_column(Column(name='velocity', data=velocity, unit='m/s'))
    table.add_column(Column(name='opacity', data=opacity))
    table.add_column(Column(name='flux', data=flux, unit='Jy', description='Flux per beam'))
    table.add_column(Column(name='sigma_opacity', data=sigma_tau, description='Noise in the absorption profile'))

    votable = from_table(table)
    writeto(votable, filename)




In [ ]:
def extract_spectrum(fname, src, continuum_start_vel, continuum_end_vel, num_edge_chan=10):

    hdulist = fits.open(fname)
    image = hdulist[0].data
    header = hdulist[0].header
    w = WCS(header)
    index = np.arange(header['NAXIS3'])
    velocities = w.wcs_pix2world(10,10,index[:],0,0)[2]

    img_slice = cube_tools.get_integrated_spectrum(image, w, src, velocities, continuum_start_vel, continuum_end_vel, radius=6)

    l_edge, r_edge = cube_tools.find_edges(img_slice, num_edge_chan)

    spectrum_array = rec.fromarrays(
        [np.arange(img_slice.size)[l_edge:r_edge],
         velocities[l_edge:r_edge],
         img_slice[l_edge:r_edge]],
        names='plane,velocity,flux')

    del image
    del header
    hdulist.close()

    return spectrum_array

def output_spectrum(spec_folder, src, spectrum_array, opacity, sigma_opacity):
    filename = '{}/{}_spec'.format(spec_folder, src['comp_name'])

    save_spectra(spectrum_array.velocity, opacity, spectrum_array.flux, filename+'.vot', sigma_opacity)
    spectrum_tools.plot_absorption_spectrum(spectrum_array.velocity, opacity, filename+'.png', tgt['comp_name'], continuum_start_vel, continuum_end_vel, sigma_opacity, range=None)
    spectrum_tools.plot_absorption_spectrum(spectrum_array.velocity, opacity, filename+'_zoom.png', tgt['comp_name'], continuum_start_vel, continuum_end_vel, sigma_opacity, range=(75,275))




In [ ]:

src_table = QTable(names=('id', 'comp_name', 'ra', 'dec', 'rating', 'flux_peak', 'mean_cont', 'sd_cont', 'opacity_range', 'max_s_max_n'),
           dtype=('int', 'U32', 'float64', 'float64', 'str', 'float64', 'float64', 'float64', 'float64', 'float64'),
            meta={'name': 'ASKAP SMC Spectra'})

print('Processing {} cutouts into spectra'.format(len(targets)))

max_spectra = 500
i = 0
for tgt in targets:
    src = get_source(file_list, tgt)
    if not src:
        print('Skipping missing src #{} {}'.format(tgt['id'], tgt['comp_name']))
        continue
    i+=1
    if i > max_spectra:
        print ("Reaching maximum spectra for this run")
        break
        
    print('\nExtracting spectrum for src #{} {}'.format(tgt['id'], tgt['comp_name']))

    plot_mom0(src['fname'], tgt['comp_name'], 'sb8906/figures', src['ra'], src['dec'], 
              src['a'], src['b'], src['pa'])
    
    continuum_start_vel = -100*u.km.to(u.m)
    continuum_end_vel = -60*u.km.to(u.m)
    spectrum = extract_spectrum(src['fname'], src, continuum_start_vel, continuum_end_vel)
    
    mean_cont, sd_cont = spectrum_tools.get_mean_continuum(spectrum.velocity, spectrum.flux, continuum_start_vel, continuum_end_vel)
    opacity = spectrum.flux/mean_cont
    sigma_opacity = sd_cont*np.ones(spectrum.velocity.shape)
    rating, opacity_range, max_s_max_n = spectrum_tools.rate_spectrum(opacity, sd_cont)

    output_spectrum('sb8906/spectra', src, spectrum, opacity, sigma_opacity)
    
    src_table.add_row([tgt['id'], tgt['comp_name'], src['ra']*u.deg, src['dec']*u.deg, 
                       rating, src['flux_peak']*u.Jy, mean_cont*u.Jy, sd_cont, opacity_range, max_s_max_n])
    
               
                                

In [ ]:
srcvotable = from_table(src_table)
writeto(srcvotable, 'askap_spectra.vot')


## Scratch area

In [ ]:
tgt = targets[0]
fname = '{}{}_sl.fits'.format(folder, tgt['comp_name'])

comp_cat_row= selavy_table[selavy_table['component_name']==tgt['comp_name']][0]
comp_cat_row
src_ra = comp_cat_row['ra_deg_cont']
src_dec = comp_cat_row['dec_deg_cont']
src_maj = comp_cat_row['maj_axis']
src_min = comp_cat_row['min_axis']
src_pa = comp_cat_row['pos_ang']
print ('Component at RA {:.4f} Dec {:.4f} is {:.2f}x{:.2f} arcsec at angle {:.1f} deg'.format(src_ra, src_dec, src_maj, src_min, src_pa))

In [ ]:
plot_mom0(fname, tgt['comp_name'], 'sb8906/figures', src_ra, src_dec, src_maj, src_min, src_pa)

In [ ]:
src = {'ra':src_ra, 'dec':src_dec, 'a':src_maj, 'b':src_min, 'pa': src_pa}

hdulist = fits.open(fname)
image = hdulist[0].data
header = hdulist[0].header
w = WCS(header)
index = np.arange(header['NAXIS3'])
#beam_maj = header['BMAJ'] * 60 * 60
#beam_min = header['BMIN'] * 60 * 60
#beam_area = math.radians(header['BMAJ']) * math.radians(header['BMIN'])
velocities = w.wcs_pix2world(10,10,index[:],0,0)[2]
continuum_start_vel = -100*u.km.to(u.m)
continuum_end_vel = -60*u.km.to(u.m)
num_edge_chan = 10

#c = SkyCoord(src_ra, src_dec, frame='icrs', unit="deg")

img_slice = cube_tools.get_integrated_spectrum(image, w, src, velocities, continuum_start_vel, continuum_end_vel, radius=6)

l_edge, r_edge = cube_tools.find_edges(img_slice, num_edge_chan)
# print("Using data range %d - %d out of %d channels." % (
#    l_edge, r_edge, len(img_slice)))

# plotSpectrum(np.arange(slice.size), slice)
spectrum_array = rec.fromarrays(
    [np.arange(img_slice.size)[l_edge:r_edge],
     velocities[l_edge:r_edge],
     img_slice[l_edge:r_edge]],
    names='plane,velocity,flux')


In [ ]:
spec_folder = 'sb8906/spectra'
filename = '{}/{}_spec'.format(spec_folder, tgt['comp_name'])
mean_cont, sd_cont = spectrum_tools.get_mean_continuum(spectrum_array.velocity, spectrum_array.flux, continuum_start_vel, continuum_end_vel)
opacity = spectrum_array.flux/mean_cont
sigma_tau = sd_cont*np.ones(spectrum_array.velocity.shape)

save_spectra(spectrum_array.velocity, opacity, spectrum_array.flux, filename+'.vot', sigma_tau)
#plt.plot(spectrum_array.flux)
spectrum_tools.plot_absorption_spectrum(spectrum_array.velocity, opacity, filename+'.png', tgt['comp_name'], continuum_start_vel, continuum_end_vel, sigma_tau, range=None)
spectrum_tools.plot_absorption_spectrum(spectrum_array.velocity, opacity, filename+'_zoom.png', tgt['comp_name'], continuum_start_vel, continuum_end_vel, sigma_tau, range=(75,275))
rating, opacity_range, max_s_max_n = spectrum_tools.rate_spectrum(opacity, sd_cont)
print ('Source {} rating: {} range:{:.2f}'.format(tgt['comp_name'], rating,opacity_range ) )

In [ ]:
src_table = QTable(names=('id', 'comp_name', 'ra', 'dec', 'rating', 'mean_cont', 'sd_cont'),
           dtype=('int', 'U32', 'float64', 'float64', 'str', 'float64', 'float64'),
            meta={'name': 'ASKAP SMC Spectra'})
#t['comp_name'].length = 32
src_table.add_row([tgt['id'], tgt['comp_name'], src_ra*u.deg, src_dec*u.deg, rating, mean_cont*u.Jy, sd_cont])

In [ ]:
good_beams = cube.identify_bad_beams(threshold=0.01)
good_beams

In [ ]:
#cube = SpectralCube.read(fname)
#print(cube)
#print (cube.beams)
m0 = cube.moment0().to(u.Jy*u.km/(u.beam*u.s))
print(m0)
m0.write('moment-0.fits', overwrite=True)
img_size = m0.shape

fig = aplpy.FITSFigure('moment-0.fits')
center_coord = fig.pixel2world(xp=img_size[0]/2, yp=img_size[1]/2)
#fig.recenter(center_coord[0], center_coord[1], radius=2*u.arcmin.to(u.deg))  # degrees
#fig.show_grayscale(stretch='log', vmin=0, vmid=-0.1)
fig.show_grayscale()#pmax=100)
fig.add_colorbar()
fig.colorbar.set_axis_label_text('Brightness (Jy km / s)')
fig.add_beam()
fig.set_theme('publication')

# Plot ellipse for each source
a_deg = src_maj*u.arcsec.to(u.deg)
b_deg = src_min*u.arcsec.to(u.deg)
fig.show_ellipses(src_ra, src_dec, a_deg, b_deg, angle=src_pa, edgecolor='red') #, 

figname = 'sb8906/figures/{}_mom0.'.format(tgt['comp_name'])
fig.savefig(figname+'.pdf')
fig.savefig(figname+'.png')


In [ ]:
# run agean to identify sources (or use preexisting)
# identify continuum velocity ranges
start_vel=(-90*u.km).to(u.m).value
end_vel=(-30*u.km).to(u.m).value

# Note that we need to make sure the beam info is set:
# sethead BMAJ='0.0047402' BMIN='0.0040442' BPA='83.183' sb8906/casda/sb8906_0029-7228_sl_*K.fits

# loop through and extract spectra for each

In [ ]:
sd_list = {}
max_list = {}
for name in file_list:
    spec, source_ids, islands = cube_tools.extract_spectra(fits_filename=name, 
                      isle_filename='0029-7228_src_isle.vot', src_filename='0029-7228_src_comp.vot',
                      continuum_start_vel=start_vel, continuum_end_vel=end_vel)
    spectrum = list(spec.values())[0]
    spec_mean, spec_sd = spectrum_tools.get_mean_continuum(spectrum.velocity, spectrum.flux, start_vel, end_vel)
    sigma_tau = np.full(spectrum.velocity.shape, spec_sd)
    print (spec_mean, np.max(spectrum.flux))
    spectrum_tools.plot_absorption_spectrum(spectrum.velocity, spectrum.flux/spec_mean, name[:-5]+'_spec.png',
                                           name, start_vel, end_vel, sigma_tau, range=(-100,250))
    
    output_spectra(spectrum.velocity, spectrum.flux/spec_mean, spectrum.flux, name[:-5]+'_spec.xml', sigma_tau)
    #plt.plot(spectrum.velocity/1000, spectrum['flux'])
    #plt.title(name)
    #plt.savefig(name[:-5]+'_spec.png')
    #plt.close()
    sd_list[name] = spec_sd
    max_list[name] = np.max(spectrum.flux/spec_mean)

In [ ]:
print (sd_list)